In [0]:
# This file trains the USTC using differential privacy 
# by adding nosie to autoencoder first layer 
# It uses a dense autoencoder. This file enables training 
# with both l1, l2 noise and both laplacian and gaussian 
# noise. See the https://arxiv.org/pdf/1802.03471.pdf for 
# details about how this works. It makes use of privacy amplifcation
# by subsampling allowing for less noise to be added
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Move to drive
%cd drive
%cd 'My Drive'
%cd Oxford-Thesis

/content/drive
/content/drive/My Drive
/content/drive/My Drive/Oxford-Thesis


In [0]:
import tensorflow as tf
import six 
import math
from tensorflow.python.training import moving_averages 
import pickle
import tensorflow.keras.backend as K
from sklearn.preprocessing import normalize
import tensorflow_probability as tfp

In [0]:
## Constants for deciding how to add differential privacy to autoencoder 
NOISE_PLACEMENT = 'img_noise'
NOISE_SCHEME = 'l2'
FILTER_SIZES = 1
NUM_FILTERS = 1 
STRIDES = 1 
L1_MEAN_SIGNAL = 0
L2_MEAN_SIGNAL = 0
L1_NOISE_SIGNAL = 0
L2_NOISE_SIGNAL = 0
EPSILON = 1.0
DELTA_DP = 0.05
SCALE = 0.1
## BECAUSE WE ARE SUBSAMPLING
SACLE = SCALE/2
EPOCHS = 40 
NOISE = 0
BATCH_SHAPE = 256
NOISE_TYPE = 'gaussian'
AUTO_ENCODER_OUT_SHAPE = 158
CONT_FEATURE_SIZE = 158
FOLDER = 'USTC2-AutoEncoderGaussianImmedPoisson'
NUM_CLASS =10
TRAIN =True 
SCALE_STRING = "0point1"

In [0]:
## DP-Mult for different types of noise 
def dp_mult( size=None):
  epsilon_dp =EPSILON
  delta_dp   = DELTA_DP
  max_pixeldp_norm = SCALE
  if NOISE_SCHEME == 'l1_l2'    or  \
      NOISE_SCHEME == 'l1_l2_s1' or  \
      NOISE_SCHEME == 'l2_l2_s1' or\
      NOISE_SCHEME == 'l2':
      return max_pixeldp_norm * \
          math.sqrt(2 * math.log(1.25 / delta_dp)) / epsilon_dp
  elif NOISE_SCHEME == 'l1_l1'  or  \
      NOISE_SCHEME == 'l1_l1_s1' or\
      NOISE_SCHEME == 'l1':
      return max_pixeldp_norm / epsilon_dp

In [0]:
## Create different types of noises for different 
## noise types and schemes
def img_dp_noise(shape):
  if NOISE_TYPE == 'laplace':
    sensitivity_multiplier = EPSILON 
    dp_mult_x = dp_mult()
    loc = tf.zeros([BATCH_SHAPE,shape])
    scale = tf.ones([BATCH_SHAPE,shape])
    epsilon = tfp.distributions.Laplace(loc,scale).sample()
    noise_scale = tf.placeholder(tf.float32,shape =(), name ='noise_scale')
    L1_SENSITIVITY = 1.0
    b = L1_SENSITIVITY *dp_mult_x*SCALE
    noise = b *epsilon
    
  elif NOISE_TYPE == 'gaussian':
    dp_mult_x = dp_mult()
    epsilon = tf.random_normal(shape=(BATCH_SHAPE,shape), mean=0,stddev=1)
    sensitivity_multiplier = EPSILON 
    L2_SENSITIVITY = 1.0
    noise_scale = tf.placeholder(tf.float32,shape =(), name ='noise_scale')
    SIGMA = tf.multiply(dp_mult_x,L2_SENSITIVITY,name= "SIGMA")
    noise_stddev = SCALE * SIGMA
    noise = noise_stddev *epsilon
  return noise

In [0]:
# Normalize the l1 direction https://github.com/columbia/pixeldp
def l1_normalize(x, dim, epsilon=1e-12, name=None):
  """Normalizes along dimension `dim` using an L1 norm.
  For a 1-D tensor with `dim = 0`, computes
      output = x / max(sum(abs(x)), epsilon)
  For `x` with more dimensions, independently normalizes each 1-D slice along
  dimension `dim`.
  Args:
    x: A `Tensor`.
    dim: Dimension along which to normalize.  A scalar or a vector of
      integers.
    epsilon: A lower bound value for the norm. Will use `sqrt(epsilon)` as the
      divisor if `norm < sqrt(epsilon)`.
    name: A name for this operation (optional).
  Returns:
    A `Tensor` with the same shape as `x`.
  """
  with tf.name_scope(name, "l1_normalize", [x]) as name:
    with tf.device("/gpu:0"):
      x          = tf.convert_to_tensor(x, name            = "x")
      abs_sum    = tf.reduce_sum(tf.abs(x), dim, keep_dims = True)
      x_inv_norm = tf.reciprocal(tf.maximum(abs_sum, epsilon))
      return tf.multiply(x, x_inv_norm, name=name)

In [0]:
with tf.device('/device:GPU:0'):
  NOISE = img_dp_noise(int(CONT_FEATURE_SIZE))
  ## Define the auto encoder without keras 
  x_ = tf.placeholder(tf.float32,shape= [BATCH_SHAPE,CONT_FEATURE_SIZE])
  y_ =tf.placeholder(tf.float32,shape= [BATCH_SHAPE,CONT_FEATURE_SIZE])
  
  x2_ =tf.placeholder(tf.float32,shape= [BATCH_SHAPE,AUTO_ENCODER_OUT_SHAPE])
  y2_ =tf.placeholder(tf.float32,shape= [BATCH_SHAPE,2])
  y3_ =tf.placeholder(tf.float32,shape= [BATCH_SHAPE,10])

In [0]:
def get_rep(x, full_rep = True):
  with tf.variable_scope("first_model",reuse=tf.AUTO_REUSE):
    with tf.device('/device:GPU:0'):
      if TRAIN == True:
        noise_image = tf.add(x,NOISE)
      else:
        noise_image = tf.add(x,img_dp_noise(CONT_FEATURE_SIZE))
      ## ADD NOISE 
      W_dense1 = tf.get_variable("dense1",initializer =tf.random_normal(stddev=0.1, shape =[CONT_FEATURE_SIZE,int(CONT_FEATURE_SIZE/2)]))
      b_dense1 =  tf.get_variable("bias1",initializer =tf.random_normal(stddev=0.1, shape =[int(CONT_FEATURE_SIZE/2)]))
      h_fc1 = tf.nn.relu(tf.matmul(tf.cast(noise_image,tf.float32),W_dense1)+b_dense1)
      
      W_dense2 =tf.get_variable("dense2",initializer =tf.random_normal(stddev=0.1,shape= [int(CONT_FEATURE_SIZE/2),int(CONT_FEATURE_SIZE/4)]))
      b_dense2 = tf.get_variable("bias2",initializer =tf.random_normal(stddev=0.1,shape= [int(CONT_FEATURE_SIZE/4)]))
      h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_dense2) + b_dense2)


      W_dense3 =tf.get_variable("dense3",initializer =tf.random_normal(stddev=0.1,shape= [int(CONT_FEATURE_SIZE/4),10]))
      b_dense3 =tf.get_variable("bias3",initializer =tf.random_normal(stddev=0.1,shape= [10])) 
      first_output = tf.matmul(h_fc2, W_dense3) + b_dense3
      h_fc3 = tf.nn.relu(first_output)

      W_dense4 =tf.get_variable("dense4",initializer =tf.random_normal(stddev=0.1,shape= [10,int(CONT_FEATURE_SIZE/4)]))
      b_dense4 =tf.get_variable("bias4",initializer =tf.random_normal(stddev=0.1,shape= [int(CONT_FEATURE_SIZE/4)]))
      h_fc4 = tf.nn.relu(tf.matmul(h_fc3, W_dense4) + b_dense4)

      W_dense5 = tf.get_variable("dense5",initializer =tf.random_normal(stddev=0.1,shape= [int(CONT_FEATURE_SIZE/4),int(CONT_FEATURE_SIZE/2)]))
      b_dense5 = tf.get_variable("bias5",initializer =tf.random_normal(stddev=0.1,shape= [int(CONT_FEATURE_SIZE/2)]))
      h_fc5 = tf.nn.relu(tf.matmul(h_fc4, W_dense5) + b_dense5)

      W_dense6 = tf.get_variable("dense6",initializer =tf.random_normal(stddev=0.1,shape= [int(CONT_FEATURE_SIZE/2),int(CONT_FEATURE_SIZE)]))
      b_dense6 = tf.get_variable("bias6",initializer =tf.random_normal(stddev=0.1,shape= [CONT_FEATURE_SIZE]))
      h_fc6 = tf.matmul(h_fc5, W_dense6) + b_dense6
      if full_rep == True:
        return h_fc6
      return first_output

In [0]:
with tf.device('/device:GPU:0'):
  mse = tf.reduce_mean(tf.losses.mean_squared_error(labels=y_, predictions=get_rep(x_,True)))
  train_step = tf.train.AdamOptimizer(1e-4,name="adam-encoder").minimize(mse)
  accuracy = mse

W0810 19:06:20.060935 139962619545472 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
import numpy as np
## Load in data
with open('./embedding/CNN_FULL/USTC_prn_mal_meta_train', 'rb') as fp:
  mal_train_meta_X = pickle.load(fp)
  fp.close()
  
with open('./embedding/CNN_FULL/USTC_prn_mal_meta_valid', 'rb') as fp:
  mal_valid_meta_X = pickle.load(fp)
  fp.close()
  
  
with open('./embedding/CNN_FULL/USTC_prn_mal_meta_train_labels', 'rb') as fp:
  mal_train_labels = pickle.load(fp)
  fp.close()
  
with open('./embedding/CNN_FULL/USTC_prn_mal_meta_valid_labels', 'rb') as fp:
  mal_valid_labels = pickle.load(fp) 
  fp.close()
  

with open('./embedding/CNN_FULL/USTC_prn_benign_meta_train', 'rb') as fp:
  benign_train_meta_X = pickle.load(fp)
  fp.close()
  
with open('./embedding/CNN_FULL/USTC_prn_benign_meta_valid', 'rb') as fp:
  benign_valid_meta_X  = pickle.load(fp)
  fp.close()
  
with open('./embedding/CNN_FULL/USTC_prn_benign_size_train', 'rb') as fp:
  benign_train_size_X = pickle.load(fp)
  fp.close()
  


In [0]:
train_labels = [[0,1]]*len(benign_train_meta_X) +[[1,0]]*len(mal_train_meta_X)
valid_labels = [[0,1]]*len(benign_valid_meta_X) +[[1,0]]*len(mal_valid_meta_X)

In [0]:
train_meta_X = np.concatenate([benign_train_meta_X,mal_train_meta_X])
valid_meta_X = np.concatenate([benign_valid_meta_X,mal_valid_meta_X])

c = list(zip(train_meta_X,train_labels))
np.random.shuffle(c)
train_meta_X,train_labels = zip(*c)

c = list(zip(valid_meta_X,valid_labels))
np.random.shuffle(c)
valid_meta_X,valid_labels = zip(*c)
train_meta_X = np.array(train_meta_X).astype(float)
valid_meta_X = np.array(valid_meta_X).astype(float)
valid_meta_X = valid_meta_X[:52992]

In [0]:
train_X =np.array(train_meta_X).astype(float)
## Subsample with lambda = 0.5 
samples = []
probs = np.random.uniform(size=len(train_X))
for index in range(len(probs)):
  if probs[index]<= 0.5:
    samples.append(train_X[index])
train_X = np.array(samples).astype(float)
valid_X =np.array(valid_meta_X).astype(float)

In [0]:
len(train_X)/256
amount = int(len(train_X)/256)*256
train_X=train_X[:amount]

In [0]:
saver = tf.train.Saver()
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
print("GPU Available: ", tf.test.is_gpu_available())
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

GPU Available:  True
Found GPU at: /device:GPU:0


In [0]:
session = tf.Session(config=config)
session.run(tf.global_variables_initializer())
with session as sess:
  noise = sess.run(NOISE)

In [0]:
with open('./DP-GRAPHS/' + FOLDER +'/NOISE'+SCALE_STRING, 'wb') as fp:
  pickle.dump(noise,fp)
  fp.close()

In [0]:
#Train the autoencoder
EPOCHS = 60
session = tf.Session(config=config)
session.run(tf.global_variables_initializer())
x_coord = 0
x_coords = []
mses = []
with session as sess:
  for epoch in range(EPOCHS):
    train_accuracy = 0 
    for batch_num in range(int(len(train_X)/BATCH_SHAPE)):
      batch = train_X[batch_num*BATCH_SHAPE:batch_num*BATCH_SHAPE+BATCH_SHAPE]
      batch_label = train_X[batch_num*BATCH_SHAPE:batch_num*BATCH_SHAPE+BATCH_SHAPE]
      train_accuracy += accuracy.eval(feed_dict={x_:batch, y_: batch_label})
      train_step.run(feed_dict={x_: batch, y_: batch_label})
    x_coords.append(x_coord)
    x_coord+=1
    mses.append(train_accuracy/(int(len(train_X)/BATCH_SHAPE)))
    print('Step %d, training accuracy %g' % (batch_num , train_accuracy/(int(len(train_X)/BATCH_SHAPE))))
    valid_acc = 0 
    for batch_num in range(int(len(valid_X)/BATCH_SHAPE)): 
      batch = valid_X[batch_num*BATCH_SHAPE:batch_num*BATCH_SHAPE+BATCH_SHAPE]
      batch_label = valid_X[batch_num*BATCH_SHAPE:batch_num*BATCH_SHAPE+BATCH_SHAPE]
      valid_acc += accuracy.eval(feed_dict={x_:batch, y_: batch_label})
    print('Step %d, validation accuracy %g' % (batch_num , valid_acc/(int(len(valid_X)/BATCH_SHAPE)))) 
  saver.save(sess, './DP-GRAPHS/' + FOLDER+ '/poisson-autoencoder'+SCALE_STRING+'.ckpt')

Step 1271, training accuracy 0.0130174
Step 206, validation accuracy 0.00272938
Step 1271, training accuracy 0.00179892
Step 206, validation accuracy 0.000898205
Step 1271, training accuracy 0.000983687
Step 206, validation accuracy 0.000627942
Step 1271, training accuracy 0.000726486
Step 206, validation accuracy 0.000463065
Step 1271, training accuracy 0.0005209
Step 206, validation accuracy 0.000344177
Step 1271, training accuracy 0.000409227
Step 206, validation accuracy 0.000286768
Step 1271, training accuracy 0.000357417
Step 206, validation accuracy 0.000256944
Step 1271, training accuracy 0.000329049
Step 206, validation accuracy 0.000239366
Step 1271, training accuracy 0.000310018
Step 206, validation accuracy 0.000225716
Step 1271, training accuracy 0.000293409
Step 206, validation accuracy 0.000212955
Step 1271, training accuracy 0.00027766
Step 206, validation accuracy 0.000200518
Step 1271, training accuracy 0.000262272
Step 206, validation accuracy 0.000188771
Step 1271, 